In [9]:
import numpy as np 
import pandas as pd 
import csv
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [10]:
train_file="../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv"
test_file="../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"

In [11]:
def parse_data_from_input(filename):
    with open(filename) as file:
        reader = csv.reader(file, delimiter=',')    
        imgs = []
        labels = []
        next(reader, None)
        for row in reader:
            label = row[0]
            data = row[1:]
            img = np.array(data).reshape((28, 28))

            imgs.append(img)
            labels.append(label)

    images = np.array(imgs).astype(float)
    labels = np.array(labels).astype(float)
    return images, labels

In [12]:
train_images, train_labels = parse_data_from_input(train_file)
test_images, test_labels = parse_data_from_input(test_file)

In [13]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [14]:
def train_val_generators(train_images, train_labels, test_images, test_labels):
    train_images = np.expand_dims(train_images, axis=3)
    test_images = np.expand_dims(test_images, axis=3)

  # Instantiate the ImageDataGenerator class and also we need to normalize pixel values and set arguments to augment the images (if desired)
    train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


  
    train_generator = train_datagen.flow(x=train_images,
                                       y=train_labels,
                                       batch_size=32) 

  

    validation_datagen = ImageDataGenerator(
    rescale=1 / 255)

 
    validation_generator = validation_datagen.flow(x=test_images,
                                                 y=test_labels,
                                                 batch_size=32) 


    return train_generator, validation_generator

In [15]:
train_generator,valid_generator = train_val_generators(train_images,train_labels, test_images, test_labels)

In [16]:
def create_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax')])
  

    model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
   
  
    return model

In [19]:
model = create_model()

In [21]:
history = model.fit(train_generator, epochs = 30, validation_data = valid_generator)